In [1]:
import sys
sys.path.append('..')
from sklearn.model_selection import train_test_split

import numpy as np
import pickle
from scipy.io import savemat
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from tqdm import tqdm
import random
import utils
import pandas as pd
import os

In [2]:
expName = 'compare'

dir = '../datasets/%s.pickle' % expName
winLENs = np.arange(0.2, 1, step=.2)
with open(dir, "rb") as fp:
    wholeset = pickle.load(fp)

sub = wholeset[-1]
tag = 'SSVEP'
# chnNames = ['PZ', 'PO5', 'POZ', 'PO4', 'PO6', 'O1', 'OZ', 'O2']
chnNames = sub['channel']
chnINX = [sub['channel'].index(i) for i in chnNames]
X = sub[tag]['X'][:, chnINX]
y = sub[tag]['y']
S = sub[tag]['STI']
sub['name']

'zhouyuqing'

In [8]:
srate = 250
winLEN = 0.3
tmin=0
tmax=0.3
targtetNUM = 40

In [9]:
from spatialFilters import TDCA
X = X-np.mean(X,axis=-1,keepdims=True)
model = TDCA(n_components=1,n_band=1,montage=targtetNUM,lag=0.14,srate=250,winLEN=winLEN)
evokeds = model.fit_transform(X,y)
X_ = model.filter(X)

In [10]:
from scipy import signal

snrs = np.zeros((40,8))
N = 75
_classes = np.unique(y)

for i,(evoked,_class) in enumerate(zip(evokeds,_classes)):
    
    epochs = X_[ y == _class ]

    for j,epoch in enumerate(epochs):

        noise = epoch-evoked
                    
        s = np.abs(np.fft.fftshift(np.fft.fft(evoked,axis=-1)/N))**2
        san = np.abs(np.fft.fftshift(np.fft.fft(epoch,axis=-1)/N))**2
        n = np.abs(np.fft.fftshift(np.fft.fft(noise, axis=-1)/N))**2

        # break
        s_power = np.sum(s[:,N//2:],axis=-1)
        all_power = np.sum(san[:,N//2:],axis=-1)
        n_power = np.sum(n[:,N//2:],axis=-1)

        snrs[i,j] = 10*np.log10(s_power/n_power)
